# 실시간 데이터 작업

In [ ]:
# 장소 리스트 불러오기
lst = pd.read_excel("/content/drive/MyDrive/DScover/2023-2 메인프/데이터/서울시 주요 113장소명 목록 (91개 수정본).xlsx")

# 각 관광지에 대한 실시간 데이터 주소
space_num = []
area_name = []
for i in range(len(lst)):
    space_num.append(str(lst.at[i,'NO']))
    area_name.append(str(lst.at[i,'AREA_NM']))
web_address = []
address_str = "http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/"

for i in range(len(lst)):
    address = address_str + space_num[i] + '/' + area_name[i]
    web_address.append(address)

web_address

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


['http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/1/강남 MICE 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/2/동대문 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/3/명동 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/4/이태원 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/5/잠실 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/6/종로·청계 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/7/홍대 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/8/경복궁',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/9/광화문·덕수궁',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/10/보신각',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata

In [ ]:
# 전체 실시간 데이터 수집
key = '6b6c58426167726b3439424b4d7873'

#전체 데이터가 담긴 리스트 생성
all_data = []

for i in range(len(lst)):
    response = requests.get(web_address[i])
    data = xmltodict.parse(response.text)

    # XML 데이터를 JSON 형식으로 변환
    json_data = json.loads(json.dumps(data))

    # JSON 데이터를 데이터프레임으로 변환
    df = pd.json_normalize(data)

    all_data.append(df)

In [ ]:
# 실시간 데이터프레임의 컬럼명과 한글 이름 데이터
column_list = pd.read_excel('/content/drive/MyDrive/DScover/2023-2 메인프/데이터/컬럼명 리스트.xlsx')
column_list = column_list.drop(['No'], axis = 1)

In [ ]:
import copy
# 원본 복사 (데이터 훼손 방지)
all_data2 = copy.deepcopy(all_data)

# all_data의 각 장소 정보를 하나의 데이터 프레임으로
merged_df = pd.DataFrame()
for i in all_data2:
    merged_df = pd.concat([merged_df, i], ignore_index=True)

# 데이터프레임 컬럼명 일부 한글화
merged_df = merged_df.drop(['SeoulRtd.citydata.list_total_count',
       'SeoulRtd.citydata.RESULT.RESULT.CODE',
       'SeoulRtd.citydata.RESULT.RESULT.MESSAGE'], axis=1)
merged_df.rename(columns={'SeoulRtd.citydata.CITYDATA.ROAD_TRAFFIC_STTS.AVG_ROAD_DATA.ROAD_TRFFIC_TIME' : '도로소통현황 업데이트 시간',
                            'SeoulRtd.citydata.CITYDATA.EVENT_STTS.EVENT_STTS' : '행사 이름'}, inplace=True)

In [ ]:
#컬럼명 전체 한글화
duplicated_columns = merged_df.columns
for j in range(len(duplicated_columns)):
    for i in range(len(column_list)):
        if merged_df.columns[j].endswith(column_list.iat[i,0]):
            merged_df.rename(columns={merged_df.columns[j]: column_list.iat[i, 1]}, inplace=True)
            break

In [ ]:
#문화 행사 정보 필터
class event_filter:
    def __init__(self, df):
        self.df = df

    #문화 행사가 존재하는 곳에 대한 필터링
    def event(self):
        event_df = self.df.loc[(self.df['문화행사명'].isna() == False) | (self.df['행사 이름'].isna() == False)].reset_index(drop=True)
        return event_df

    def __del__(self):
        pass

#주차장 정보 필터
class parkinglot_filter:
    def __init__(self, df):
        self.df = df

    def parking_lot(self):
        parking_lot_df = self.df.loc[(self.df['주차장명'].isna() == False)].reset_index(drop=True)

        return parking_lot_df

    def __del__(self):
        pass

class complexity_filter:
    def __init__(self,df):
        self.df = df

# 장소 혼잡도 메세지에 따라 필터링
    def complexity(self):
        complexity_df = self.df.loc[(self.df['장소 혼잡도 지표 관련 메세지'] == '사람이 몰려있을 가능성이 낮고 붐빔은 거의 느껴지지 않아요. 도보 이동이 자유로워요.')
        | (self.df['장소 혼잡도 지표 관련 메세지'] == '사람이 몰려있을 수 있지만 크게 붐비지는 않아요. 도보 이동에 큰 제약이 없어요.')].reset_index(drop=True)

        return complexity_df

In [ ]:
# 필터링 후 정보 출력 : print_info() 클래스 설정

class print_info:
    def __init__(self, df, option):
        self.df = df
        self.option = option

    def display_info(self):
        for i in range(len(self.df)):
            print("핫스팟 장소명: {}".format(self.df.iloc[i]['핫스팟 장소명']))

            # 날씨 정보 프린트
            print('날씨 정보:')
            print('\t강수관련정보: {}'.format(self.df.iloc[i]['강수관련 메세지']))
            print('\t자외선메세지: {}'.format(self.df.iloc[i]['자외선메세지']))
            print('\t미세먼지/초미세먼지 지표: {}'.format(self.df.iloc[i]['미세먼지지표'], self.df.iloc[i]['초미세먼지지표']))

            # 문화 행사 출력
            if self.option[0] == 1:
                # 문화행사명이 NaN이 아닌 경우에만 출력
                if pd.notna(self.df.iloc[i]['문화행사명']):
                    print('\t문화행사명: {}'.format(self.df.iloc[i]['문화행사명']))
                # 행사 이름이 리스트 형태인 경우에만 반복
                if '행사 이름' in self.df.columns and isinstance(self.df.iloc[i]['행사 이름'], list):
                    for event in self.df.iloc[i]['행사 이름']:
                        print('\t문화행사명 및 장소: {}\t(위치: {})'.format(event['EVENT_NM'], event['EVENT_PLACE']))

            # 주차장 정보 출력
            if self.option[1] == 1 and pd.notna(self.df.iloc[i]['주차장명']):
                print('\n\t주차장 정보:')
                print('\t주차장명: {}'.format(self.df.iloc[i]['주차장명']))
                print('\t현재수용가능명: {}'.format(self.df.iloc[i]['주차장 수용 가능 면수']))
                print('\t유무료 여부: {}'.format(self.df.iloc[i]['유무료 여부']))

            # 혼잡도 정보 출력
            if self.option[2] == 1 and pd.notna(self.df.iloc[i]['장소 혼잡도 지표 관련 메세지']):
                print("\n\t혼잡도 정보: {}".format(self.df.iloc[i]['장소 혼잡도 지표 관련 메세지']))

            print('\n')

In [ ]:
class execute_filtering:

    def __init__(self, df, options):
        self.df = df
        self.options = options

    def filtering(self):
        # 데이터 필터링 전 원본 복사
        filtered_df = copy.deepcopy(self.df)

        for option in range(len(self.options)):
            if self.options[option] == 1 :

                #행사 정보 필터링
                if option == 0:
                    filtered_df = event_filter(filtered_df).event()

                # 주차장 정보 기반 필터링
                elif option == 1:
                    filtered_df = parkinglot_filter(filtered_df).parking_lot()

                # 혼잡도 정보 기반 필터링
                elif option == 2:
                    filtered_df = complexity_filter(filtered_df).complexity()

        return filtered_df

In [ ]:
# 필터링 조건 설정 및 결과 테스트
event_opt = 1
parkinglot_opt = 1
complexity_opt = 1

options = [event_opt, parkinglot_opt, complexity_opt]

result = execute_filtering(merged_df, options)
filtered_result = result.filtering()